In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
real_news = pd.read_csv(r"D:\project\fake_nwes\True.csv")  
fake_news = pd.read_csv(r"D:\project\fake_nwes\Fake.csv")  

In [ ]:
real_news["class"] = 0
fake_news["class"] = 1

In [ ]:
data = pd.concat([real_news, fake_news], ignore_index=True)

In [ ]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
y = data["class"].values
X = data["text"].values

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [ ]:
X = [clean_text(text) for text in X]

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=300)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=300))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10


c:\Program Files\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


562/562 ━━━━━━━━━━━━━━━━━━━━ 43s 73ms/step - accuracy: 0.8757 - loss: 0.2743 - val_accuracy: 0.9865 - val_loss: 0.0437
Epoch 2/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 46s 82ms/step - accuracy: 0.9886 - loss: 0.0327 - val_accuracy: 0.9927 - val_loss: 0.0251
Epoch 3/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 60s 107ms/step - accuracy: 0.9960 - loss: 0.0137 - val_accuracy: 0.9933 - val_loss: 0.0238
Epoch 4/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 42s 75ms/step - accuracy: 0.9982 - loss: 0.0067 - val_accuracy: 0.9800 - val_loss: 0.0681
Epoch 5/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 86s 154ms/step - accuracy: 0.9915 - loss: 0.0298 - val_accuracy: 0.9940 - val_loss: 0.0235
Epoch 6/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 70s 125ms/step - accuracy: 0.9989 - loss: 0.0047 - val_accuracy: 0.9939 - val_loss: 0.0224
Epoch 7/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 40s 70ms/step - accuracy: 0.9993 - loss: 0.0023 - val_accuracy: 0.9951 - val_loss: 0.0189
Epoch 8/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 41s 73ms/step - accuracy: 0.9999 - loss: 0.0011 - val_accu

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Prediction: REAL NEWS


In [ ]:
MODEL_PATH = "fake_news_lstm_model.h5"
TOKENIZER_PATH = "tokenizer.pkl"
model.save(MODEL_PATH)
with open(TOKENIZER_PATH, "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
import numpy as np
import re
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

MODEL_PATH = "fake_news_lstm_model.h5"
TOKENIZER_PATH = "tokenizer.pkl"

model = load_model(MODEL_PATH)
with open(TOKENIZER_PATH, "rb") as handle:
    tokenizer = pickle.load(handle)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

def predict_news(news_text):
    cleaned_text = clean_text(news_text)
    seq = tokenizer.texts_to_sequences([cleaned_text])
    padded_seq = pad_sequences(seq, maxlen=300)
    prediction = model.predict(padded_seq)
    return "FAKE NEWS" if prediction > 0.5 else "REAL NEWS"

if __name__ == "__main__":
    user_input = input("Enter a news headline or article: ")
    result = predict_news(user_input)
    print(f"Prediction: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
Prediction: FAKE NEWS
